In [ ]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections

In [ ]:
class EmbeddingDot:
    def __init__(self,W):
        self.embed=Embedding(W)
        self.params=self.embed.params
        self.grads=self.embed.grads

    def forward(self,h,idx):
        target_W=self.embed.forward(idx)
        # print('target_W\n',target_W)
        out=np.sum(target_W*h,axis=1)
        self.cache=(h,target_W)
        return out

    def backward(self,dout):
        h,target_W=self.cache
        dout=dout.reshape(dout.shape[0],1)  #dtarget_Wh=np.ones(dout.shape[0],1)*dout
        dtarget_W=dout*h
        self.embed.backward(dtarget_W)
        dh=dout*target_W
        return dh

In [ ]:
class UnigramSampler:
    '''
    vocab_size(語彙数)のカウント、negative_sample(負例)の選択確率の導出、負例の選択を行う
    '''
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        #vocab_size(語彙数)をカウントする
        counts = collections.Counter()  #配列の各要素の個数をカウントして辞書化するクラス{you:3,say:4,...}
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        #各語彙の確率分布(word_p)を求める
        self.word_p = np.zeros(vocab_size)  #語彙数分の行列を用意する
        for i in range(vocab_size):
            self.word_p[i] = counts[i]  #各語彙のカウントした数を代入する

        self.word_p = np.power(self.word_p, power)  #各数のpower乗をとる.低い確率を少し底上げする（例：power=0.75）
        self.word_p /= np.sum(self.word_p)  #power乗した個数の総和で割り、確率化する

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()  #確率分布配列を深いコピー
                target_idx = target[i]  #正例のミニバッチのi番目の要素（word_id）をtarget_idxに代入
                p[target_idx] = 0   #正例の確率を０にする
                p /= p.sum()    #各確率を負例の選択確率として再計算
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)    #確率分布に従って負例を選択する
        else:
            # GPU(cupy）で計算するときは、速度を優先
            # 負例にターゲットが含まれるケースがある
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [ ]:
class NegativeSamplingLoss:
    def __init__(self,W,corpus,power=0.75,sample_size=5):
        self.sample_size=sample_size
        self.sampler=UnigramSampler(corpus,power,sample_size)
        self.loss_layers=[SigmoidWithLoss() for _ in range(sample_size+1)]   #負例サンプルサイズ＋正例用のレイヤひとつを生成
        self.embed_dot_layers=[EmbeddingDot(W) for _ in range(sample_size+1)]   #負例サンプルサイズ＋正例用のレイヤひとつを生成

        self.params,self.grads=[],[]
        for layer in self.embed_dot_layers:
            self.params+=layer.params
            self.grads+=layer.grads

    def forward(self,h,target):
        batch_size=target.shape[0]
        negative_sample=self.sampler.get_negative_sample(target)

        #正例のフォワード
        score=self.embed_dot_layers[0].forward(h,target)    #最初のレイヤを正例用のレイヤとして使用する
        correct_label=np.ones(batch_size,dtype=np.int32)    #正解ラベル1をバッチ数だけ用意する
        loss=self.loss_layers[0].forward(score,correct_label)
        
        #負例のフォワード
        negative_label=np.zeros(batch_size,dtype=np.int32)  #不正解ラベル０をバッチ数だけ用意する
        for i in range(self.sample_size):
            negative_target=negative_sample[:,i]
            score=self.embed_dot_layers[1+i].forward(h,negative_target)
            loss+=self.loss_layers[1+i].forward(score,negative_label)

        return loss

    def backward(self,dout=1):
        dh=0
        for l0,l1 in zip(self.loss_layers,self.embed_dot_layers):
            dscore=l0.backward(dout)
            dh+=l1.backward(dscore)

        return dh